In [57]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('json')
import datetime as dt
from sublimpy import utils
import pytz

# Open SOS Measurement Dataset

In [58]:
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet')

In [59]:
tidy_df = utils.modify_df_timezone(
    tidy_df,
    pytz.UTC,
    'US/Mountain',   
)
tidy_df = tidy_df.set_index('time').sort_index().loc['20221130':'20230508'].reset_index()

In [60]:
# quick way to get variable info if we want it 
# import xarray as xr
# ds = xr.open_dataset("/storage/elilouis/sublimationofsnow/sosnoqc/isfs_20221228.nc")
# ds['SWE_p2_c']

# Make plot showing daily sublimation by clear/blowing snow 

## Calculate daily sublimation

In [61]:
import metpy.constants
seconds_per_timestep = 60*30

In [62]:
daily_sub_by_blowingsnow_src = tidy_df[
    tidy_df.variable.isin(['w_h2o__3m_c', 'SF_avg_1m_ue', 'SF_avg_2m_ue'])
].pivot_table(
    values = 'value',
    index = 'time',
    columns = 'variable'
)
daily_sub_by_blowingsnow_src['SF_avg_max_ue'] = daily_sub_by_blowingsnow_src[['SF_avg_1m_ue', 'SF_avg_2m_ue']].max(axis=1)
daily_sub_by_blowingsnow_src['blowing snow'] = daily_sub_by_blowingsnow_src['SF_avg_max_ue'] > 0
daily_sub_by_blowingsnow_src['Sublimation (mm)'] = daily_sub_by_blowingsnow_src[
    'w_h2o__3m_c'
]*seconds_per_timestep / metpy.constants.density_water.magnitude# calculate daily sublimation
daily_sub_by_blowingsnow_src = daily_sub_by_blowingsnow_src.groupby([pd.Grouper(freq='1440Min'), 'blowing snow']).sum()
daily_sub_by_blowingsnow_src = daily_sub_by_blowingsnow_src.reset_index()
daily_sub_by_blowingsnow_src

variable,time,blowing snow,SF_avg_1m_ue,SF_avg_2m_ue,w_h2o__3m_c,SF_avg_max_ue,Sublimation (mm)
0,2022-11-30,False,0.000000,0.000000,-0.020202,0.000000,-0.036364
1,2022-12-01,False,0.000000,0.000000,0.064220,0.000000,0.115599
2,2022-12-02,False,0.000000,0.000000,0.001645,0.000000,0.002961
3,2022-12-02,True,3.015333,3.996667,0.201485,4.093667,0.362683
4,2022-12-03,False,0.000000,0.000000,0.013821,0.000000,0.024879
...,...,...,...,...,...,...,...
261,2023-05-05,True,0.000000,0.001667,0.010753,0.001667,0.019355
262,2023-05-06,False,0.000000,0.000000,0.369998,0.000000,0.666013
263,2023-05-06,True,0.063333,0.121667,0.002033,0.123333,0.003659
264,2023-05-07,False,0.000000,0.000000,0.127232,0.000000,0.229023


## Plot

In [63]:
daily_sub_chart = alt.Chart(daily_sub_by_blowingsnow_src).mark_bar(width=2.5).encode(
    alt.X("time:T", title=None),
    alt.Y("Sublimation (mm):Q").title("Daily sublimation (mm)"), 
    alt.Color("blowing snow:N").scale(domain=[True, False], range=['#000000', '#C0C0C0']),
    tooltip='time:T'
).properties(height = 100, width=500)

In [64]:
daily_sub_chart.configure_axis(grid=False)

alt.Chart(...)

# Make stability regime hovmoller plot

## Using temp gradient (pot virtal temp gradient)

In [65]:
tidy_df.query("variable == 'temp_gradient_3m_c'").head(1015)
tidy_df.query("variable == 'Ri_3m_c'").head(1015)

,time,variable,value,height,tower,measurement
458,2022-11-30 00:00:00,Ri_3m_c,0.095122,3.0,c,richardson number
1225,2022-11-30 00:30:00,Ri_3m_c,1.681547,3.0,c,richardson number
1455,2022-11-30 01:00:00,Ri_3m_c,0.376380,3.0,c,richardson number
2234,2022-11-30 01:30:00,Ri_3m_c,0.118442,3.0,c,richardson number
3139,2022-11-30 02:00:00,Ri_3m_c,0.830208,3.0,c,richardson number
...,...,...,...,...,...,...
671241,2022-12-21 01:00:00,Ri_3m_c,0.494114,3.0,c,richardson number
671344,2022-12-21 01:30:00,Ri_3m_c,0.311829,3.0,c,richardson number
672501,2022-12-21 02:00:00,Ri_3m_c,0.239432,3.0,c,richardson number
672971,2022-12-21 02:30:00,Ri_3m_c,0.416677,3.0,c,richardson number


In [66]:
src = tidy_df.query("variable == 'temp_gradient_3m_c'")
src = src.assign(hour = src.time.dt.hour)
src = src.assign(hour_end = src.time.dt.hour + 1)
src = src.assign(start_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0)))
src = src.assign(end_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0) + dt.timedelta(hours=24)))
src = src.groupby([
    'hour',
    'hour_end',
    'start_date',
    'end_date',
])[['value']].median().reset_index()

# this makes it easy to make a custom colorscale with 
# negative values as a drastically different color
# round for values that are equal to or greater than 0 (stable cases)
stable_src = src[src.value >= -0.01]
stable_src['value'] = stable_src['value'].round(1)

unstable_src = src[src.value < -0.01]
unstable_src['value'] = -0.1

src = pd.concat([stable_src, unstable_src])

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_86272/836200928.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stable_src['value'] = stable_src['value'].round(1)
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_86272/836200928.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unstable_src['value'] = -0.1


In [67]:
domain = np.linspace(-0.1,0.8, 10).round(1)
range = [
    '#ff2600', #red
    '#ffffff', #blue scale, increasinlgy blue
    '#e6ebfc',
    '#cbd8f9',
    '#aec4f6',
    '#90b0f4',
    '#7199f3',
    '#4f80f2',
    '#2a62f4',
    '#0433ff',    
]
tempgrad_hovmoller_plot = alt.Chart(src.dropna()).mark_rect().encode(
        alt.X("start_date:T").title("date"),
        alt.X2("end_date:T"),
        alt.Y("hour:Q").title(['hour of day']).scale(reverse=True),
        alt.Y2("hour_end:Q"),
        alt.Color("value:O").scale(domain=domain, range=range).title('dθᵥ /dz (˚C/m)')
).properties(height = 100, width=500)
tempgrad_hovmoller_plot

alt.Chart(...)

## Using Ri_3m_c

In [68]:
src = tidy_df.query("variable == 'Ri_3m_c'")
src = src.assign(hour = src.time.dt.hour)
src = src.assign(hour_end = src.time.dt.hour + 1)
src = src.assign(start_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0)))
src = src.assign(end_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0) + dt.timedelta(hours=24)))
src = src.groupby([
    'hour',
    'hour_end',
    'start_date',
    'end_date',
])[['value']].median().reset_index()
# this makes it easy to make a custom colorscale with 
# negative values as a drastically different color
src['value'] = src['value'].round(1)

In [69]:
alt.Chart(src.dropna()).mark_rect().encode(
        alt.X("start_date:T"),
        alt.X2("end_date:T"),
        alt.Y("hour:Q"),
        alt.Y2("hour_end:Q"),
        alt.Color("value:Q").scale(scheme='blueorange', domain=[-1,1.0])
).properties(height = 100, width=500).configure_axis(grid=False)

alt.Chart(...)

## Using $\Omega_{3m}^c$

In [70]:
src = tidy_df.query("variable == 'omega_3m_c'")
src = src.assign(hour = src.time.dt.hour)
src = src.assign(hour_end = src.time.dt.hour + 1)
src = src.assign(start_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0)))
src = src.assign(end_date = src.time.apply(lambda dt_p: dt_p.replace(hour=0,minute=0,second=0) + dt.timedelta(hours=24)))
src = src.groupby([
    'hour',
    'hour_end',
    'start_date',
    'end_date',
])[['value']].median().reset_index()
# this makes it easy to make a custom colorscale with 
# negative values as a drastically different color
src['value'] = src['value'].round(1)

In [71]:
alt.Chart(src.dropna()).mark_rect().encode(
        alt.X("start_date:T"),
        alt.X2("end_date:T"),
        alt.Y("hour:Q"),
        alt.Y2("hour_end:Q"),
        alt.Color("value:Q").scale(scheme='blueorange', domain=[0, .86], clamp=True),
).properties(height = 100, width=500).configure_axis(grid=False)

alt.Chart(...)

In [72]:
src = tidy_df[tidy_df.variable.isin([
    'omega_3m_c',
    'Ri_3m_c',
    'temp_gradient_3m_c',
])][['time', 'variable', 'value']].pivot(index='time', columns='variable')
src.columns = src.columns.droplevel(0)

coupled_rule = alt.Chart(pd.DataFrame({'x':[0.43]})).mark_rule().encode(y='x')

base_chart = alt.Chart(
    src.groupby(pd.Grouper(freq='60Min')).median()
).mark_circle(
    size=5,
    opacity=0.25
)
alt.Chart(
    src.groupby(pd.Grouper(freq='60Min')).median()
).mark_circle(
    size=10,
    opacity=0.25
).properties(width = 200, height = 200)
omega_yaxis = alt.Y('omega_3m_c:Q').scale(domain=[0,5], clamp=True)
ri_xaxis = alt.X("Ri_3m_c:Q").scale(domain=[-0.5,0.5], clamp=True)

omega_vs_ri = base_chart.encode(
    ri_xaxis,
    omega_yaxis,
) + coupled_rule
tempgrad_vs_ri = base_chart.encode(
    ri_xaxis,
    alt.Y('temp_gradient_3m_c:Q'),
)
omega_vs_tempgrad = base_chart.encode(
    alt.X("temp_gradient_3m_c:Q"),
    omega_yaxis,
) + coupled_rule
omega_vs_ri | tempgrad_vs_ri | omega_vs_tempgrad

alt.HConcatChart(...)

# Make stability regime time series plot

In [73]:
def temp_gradient_to_stability_regime(x):
    if np.isnan(x):
        return None
    elif x < -0.01:
        return "unstable"
    elif x >= -0.01 and x <= 0.01:
        return "neutral"
    elif x > 0.01:
        return "stable"
    else:
        raise ValueError("what?")
src = tidy_df.query("variable == 'temp_gradient_3m_c'")
src['date'] = src.time.dt.date
src[src.time.dt.hour.isin([0,1])]
src_day = src[src.time.dt.hour.isin([12,13])].assign(time_of_day = 'day')
src_night = src[src.time.dt.hour.isin([0,1])].assign(time_of_day = 'night')
src = pd.concat([src_day, src_night])
src = src.groupby(['date', 'time_of_day']).mean(numeric_only=True).reset_index()
src['stability regime'] = src['value'].apply(temp_gradient_to_stability_regime)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_86272/2887523614.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  src['date'] = src.time.dt.date


In [74]:
stability_regime_chart = alt.Chart(src).mark_bar().encode(
    alt.X("date:T"),
    alt.Color("stability regime:N").scale(domain = ['neutral', 'stable', 'unstable'], range=['#000000', '#1f77b4', '#ff7f0e']  ),
    alt.Y("time_of_day:N", sort=['night', 'day'], title=None)
).properties(width = 500)

In [75]:
src = tidy_df[tidy_df.measurement.isin([
    'potential virtual temperature',
    'surface potential virtual temperature'
])]
src = src.set_index("time").groupby([pd.Grouper(freq='60Min'), 'height']).mean(numeric_only=True).reset_index()
src['hour'] = src.time.dt.hour
src = src[src['hour']%4 == 0]

def profile_chart(src):
    return alt.Chart(src).mark_line().encode(
        alt.X("value:Q").sort('-y').title("Potential virtual temperature (˚C)"),
        alt.Y("height:Q"),
        alt.Color('hour:O').scale(scheme='rainbow')
    ).properties(width = 200, height = 150)

In [76]:
# profile_chart(src[src.time.dt.date == dt.date(2022, 12, 22)]).properties(title = "Blowing snow day (2022-12-22)") |\
profiles_chart = profile_chart(src[src.time.dt.date == dt.date(2023, 2, 11)]).properties(title = "Sunny, midwinter day (2023-02-11)") |\
profile_chart(src[src.time.dt.date == dt.date(2023, 3, 28)]).properties(title = "Sunny, spring day (2023-03-28)")

In [77]:
(
    (daily_sub_chart & stability_regime_chart).resolve_scale(color='independent')
    &
    profiles_chart
)

alt.VConcatChart(...)

In [78]:
seasonal_conditions_figure = (
    (daily_sub_chart & tempgrad_hovmoller_plot).resolve_scale(color='independent', x='shared')
    &
    profiles_chart
)
seasonal_conditions_figure.save("seasonal_conditions_figure.png", ppi=200)
seasonal_conditions_figure

alt.VConcatChart(...)

# Calculate how frequently blowing snow occurred

In [ ]:
[v for v in tidy_df.variable.unique() if v.startswith('SF')] 

In [ ]:
# calculate blowing snow flux as the sum of the two sensors, 
blowing_snow_src = tidy_df[
    tidy_df.variable.isin(['SF_avg_2m_ue', 'SF_avg_1m_ue'])
].pivot(
    columns = 'variable',
    index = 'time',
    values = ['value']
)

blowing_snow_src.columns = blowing_snow_src.columns.droplevel(0)
blowing_snow_src['SF_avg_ue'] = blowing_snow_src['SF_avg_1m_ue'] + blowing_snow_src['SF_avg_2m_ue']

In [ ]:
fraction_time_with_blowing_snow = len( 
    blowing_snow_src.query("SF_avg_ue > 0")
) / len(
    blowing_snow_src
)

print(f"Blowing snow occurred {round(fraction_time_with_blowing_snow*100, 1)}% of the time")

# Calculate how much sublimation occurred during blowing snow

In [ ]:
times_with_blowing_snow = blowing_snow_src.query("SF_avg_ue > 0").index.values

min_lh_fluxes_during_bs = tidy_df.query("variable == 'w_h2o__3m_c'")
min_lh_fluxes_during_bs = min_lh_fluxes_during_bs[min_lh_fluxes_during_bs.time.isin(times_with_blowing_snow)]
min_lh_fluxes_no_bs = tidy_df.query("variable == 'w_h2o__3m_c'")
min_lh_fluxes_no_bs = min_lh_fluxes_no_bs[ ~ min_lh_fluxes_no_bs.time.isin(times_with_blowing_snow)]

mm_sublimation_during_bs = (min_lh_fluxes_during_bs['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_no_bs = (min_lh_fluxes_no_bs['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_total = (mm_sublimation_during_bs + mm_sublimation_no_bs)

print("total sublimaton at 3m_c")
print(mm_sublimation_total)
print("during bs sublimaton at 3m_c, not during bs")
print(mm_sublimation_during_bs, mm_sublimation_no_bs)
print("respective percentages")
print(
    round(100*mm_sublimation_during_bs/mm_sublimation_total, 1), 
    round(100*mm_sublimation_no_bs/mm_sublimation_total, 1)
)


# Calculate how frequently different stability regimes occurred

## Using static stability, $\frac{d\theta_v}{dz}$ at 3m 

In [ ]:
def temp_gradient_to_stability_regime(x, threshold = 0.01):
    if np.isnan(x):
        return None
    elif x < -threshold:
        return "unstable"
    elif x >= -threshold and x <= threshold:
        return "neutral"
    elif x > threshold:
        return "stable"
    else:
        raise ValueError("what?")

In [ ]:
# calculate blowing snow flux as the sum of the two sensors, 
stab_regimes_src = tidy_df[
    tidy_df.variable == 'temp_gradient_3m_c'
].pivot(
    columns = 'variable',
    index = 'time',
    values = ['value']
)

stab_regimes_src.columns = stab_regimes_src.columns.droplevel(0)
stab_regimes_src['stab_regime_3m_c'] = stab_regimes_src['temp_gradient_3m_c'].apply(lambda v: temp_gradient_to_stability_regime(v, 0.01))
print(stab_regimes_src['stab_regime_3m_c'].value_counts()/len(stab_regimes_src['stab_regime_3m_c'].dropna()))

## Using dynamic stability, $Ri$

In [ ]:
def ri_to_stability_regime(x, unstable_threshold = -0.01, stable_threshold=0.25):
    if np.isnan(x):
        return None
    elif x < unstable_threshold:
        return "unstable"
    elif x >= unstable_threshold and x <= stable_threshold:
        return "neutral"
    elif x > stable_threshold:
        return "stable"
    else:
        raise ValueError("what?")

In [ ]:
# calculate blowing snow flux as the sum of the two sensors, 
dynamicstab_regimes_src = tidy_df[
    tidy_df.variable == 'Ri_3m_c'
].pivot(
    columns = 'variable',
    index = 'time',
    values = ['value']
)

dynamicstab_regimes_src.columns = dynamicstab_regimes_src.columns.droplevel(0)
dynamicstab_regimes_src['stab_regime_3m_c'] = dynamicstab_regimes_src['Ri_3m_c'].apply(lambda v: ri_to_stability_regime(v, 0.01))
print(dynamicstab_regimes_src['stab_regime_3m_c'].value_counts()/len(dynamicstab_regimes_src['stab_regime_3m_c'].dropna()))

## Using coupling parameter, $\Omega$

In [ ]:
def omega_to_coupling_regime(x, weakly_coupled_threshold = 0.43, strongly_coupled_threshold=0.61):
    if np.isnan(x):
        return None
    elif x < weakly_coupled_threshold:
        return "uncoupled"
    elif x >= weakly_coupled_threshold and x <= strongly_coupled_threshold:
        return "weakly coupled"
    elif x > strongly_coupled_threshold:
        return "strongly coupled"
    else:
        raise ValueError("what?")

In [ ]:
# calculate blowing snow flux as the sum of the two sensors, 
coupling_regimes_src = tidy_df[
    tidy_df.variable == 'omega_3m_c'
].pivot(
    columns = 'variable',
    index = 'time',
    values = ['value']
)

coupling_regimes_src.columns = coupling_regimes_src.columns.droplevel(0)
coupling_regimes_src['omega_regime_3m_c'] = coupling_regimes_src['omega_3m_c'].apply(lambda v: omega_to_coupling_regime(v))
print(coupling_regimes_src['omega_regime_3m_c'].value_counts()/len(coupling_regimes_src['omega_regime_3m_c'].dropna()))

# Calculate how much sublimation occured during different stability regimes

## static stability

In [ ]:
times_w_stable = stab_regimes_src.query("stab_regime_3m_c == 'stable'").index.values
times_w_unstable = stab_regimes_src.query("stab_regime_3m_c == 'unstable'").index.values
times_w_neutral = stab_regimes_src.query("stab_regime_3m_c == 'neutral'").index.values

lhfluxes = tidy_df.query("variable == 'w_h2o__3m_c'")

lhdluxes_stable = lhfluxes[lhfluxes.time.isin(times_w_stable)]
lhdluxes_unstable = lhfluxes[lhfluxes.time.isin(times_w_unstable)]
lhdluxes_neutral = lhfluxes[lhfluxes.time.isin(times_w_neutral)]

In [ ]:
mm_sublimation_stable = (lhdluxes_stable['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_unstable = (lhdluxes_unstable['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_neutral = (lhdluxes_neutral['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_total = mm_sublimation_stable + mm_sublimation_unstable + mm_sublimation_neutral
print('stable', 'neutral', 'unstable')
print(mm_sublimation_stable, mm_sublimation_neutral, mm_sublimation_unstable)
print(
    mm_sublimation_stable/mm_sublimation_total, 
    mm_sublimation_neutral/mm_sublimation_total,
    mm_sublimation_unstable/mm_sublimation_total, 
)

# dynamic stability

In [ ]:
times_w_stable = dynamicstab_regimes_src.query("stab_regime_3m_c == 'stable'").index.values
times_w_unstable = dynamicstab_regimes_src.query("stab_regime_3m_c == 'unstable'").index.values
times_w_neutral = dynamicstab_regimes_src.query("stab_regime_3m_c == 'neutral'").index.values

lhfluxes = tidy_df.query("variable == 'w_h2o__3m_c'")

lhdluxes_stable = lhfluxes[lhfluxes.time.isin(times_w_stable)]
lhdluxes_unstable = lhfluxes[lhfluxes.time.isin(times_w_unstable)]
lhdluxes_neutral = lhfluxes[lhfluxes.time.isin(times_w_neutral)]

In [ ]:
mm_sublimation_stable = (lhdluxes_stable['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_unstable = (lhdluxes_unstable['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_neutral = (lhdluxes_neutral['value']*seconds_per_timestep / metpy.constants.density_water.magnitude).sum()
mm_sublimation_total = mm_sublimation_stable + mm_sublimation_unstable + mm_sublimation_neutral
print('stable', 'neutral', 'unstable')
print(mm_sublimation_stable, mm_sublimation_neutral, mm_sublimation_unstable)
print(
    mm_sublimation_stable/mm_sublimation_total, 
    mm_sublimation_neutral/mm_sublimation_total,
    mm_sublimation_unstable/mm_sublimation_total, 
)